In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor, AbsCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

use_relax = True
use_solid_steady = False

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFDSNavg%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, steady=use_solid_steady, time_order=2)
solverS.output_file = 'solid_outputs/resFDSNavg%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_flux_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
fluxF = InterfaceData(size=inodes, value=0.0)
fluxS = InterfaceData(size=inodes, value=0.0)
tempF = InterfaceData(size=inodes, value=300.0)
tempS = InterfaceData(size=inodes, value=T_bot)
thetaT = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = AbsCovergenceMonitor(tol=1e-4)

# maximum pc steps allowed
max_pc_steps = 200

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
Nmax = 8000
t = 0.0
dt = 0.05
avg_pc_iterations = 0.0
flog = open('FDSNavg%i.log' % k, mode='w')
flog.write('Fluid Dirichlet with solid Neumann setting, study case: %i\n' % k)
flog.close()
flog = open('FDSNavg%i.log' % k, mode='a')

temps_prev = tempS.curr.copy()
fluxf_prev = fluxF.curr.copy()

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    temps_prev[:] = tempS.curr
    fluxf_prev[:] = fluxF.curr
    
    # mapping
    tempF.curr[:] = tempS.curr[s2f]
    
    solverF.put_temperature_interface(
        grid=interface_grid,
        side=interface_side,
        axis=interface_axis,
        data=tempF.curr
    )
    
    while True:
        # back up previous interface value
        tempF.backup()
        fluxF.backup()
        tempS.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid interface flux
        fluxF.curr[:] = solverF.get_heat_flux_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        fluxS.curr[:] = 0.5*(fluxF.curr[f2s]+fluxf_prev[f2s])
        # update solid flux interface
        solverS.update_flux_interface(interface_tag, fluxS.curr)
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface temperature
        tempS.curr[:] = solverS.get_interface_temperature(tag=interface_tag)
        
        # mapping
        tempF.curr[:] = tempS.curr[s2f]
        
        # update residual
        tempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(tempF)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(tempF)
                under_relax.update_solution(tempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=0.5*(tempF.curr+temps_prev[s2f])
            )
            solverF.restore_previous_solutions()
            solverS.restore()
            pc_counts = pc_counts + 1

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        solverS.write()
        fluid_ofile.write(solverF, t)
    thetaT.backup()
    thetaT.curr[:] = (solverF.get_temperature_interface(3,0,1)-300)/10
    thetaT.update_res()
    is_conv = conv_solution.determine_convergence(thetaT)
    if is_conv:
        if step % 50 and step != Nmax-1:
            solverS.write()
            fluid_ofile.write(solverF, t)
        break
    
flog.close()

step=0, pc_iterations=8.
step=1, pc_iterations=4.
step=2, pc_iterations=4.
step=3, pc_iterations=3.
step=4, pc_iterations=2.
step=5, pc_iterations=2.
step=6, pc_iterations=2.
step=7, pc_iterations=2.
step=8, pc_iterations=2.
step=9, pc_iterations=2.
step=10, pc_iterations=2.
step=11, pc_iterations=2.
step=12, pc_iterations=2.
step=13, pc_iterations=2.
step=14, pc_iterations=2.
step=15, pc_iterations=2.
step=16, pc_iterations=2.
step=17, pc_iterations=2.
step=18, pc_iterations=2.
step=19, pc_iterations=2.
step=20, pc_iterations=2.
step=21, pc_iterations=2.
step=22, pc_iterations=2.
step=23, pc_iterations=2.
step=24, pc_iterations=2.
step=25, pc_iterations=2.
step=26, pc_iterations=2.
step=27, pc_iterations=2.
step=28, pc_iterations=2.
step=29, pc_iterations=2.
step=30, pc_iterations=2.
step=31, pc_iterations=2.
step=32, pc_iterations=2.
step=33, pc_iterations=2.
step=34, pc_iterations=2.
step=35, pc_iterations=2.
step=36, pc_iterations=2.
step=37, pc_iterations=2.
step=38, pc_iterations

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat(
    'numeric_res/kFDSN_avg%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.91518115 0.92192822 0.92678287 0.93074845 0.93411877 0.93697422
 0.9395174  0.94179208 0.94387006 0.94582422 0.94761339 0.94929775
 0.95086967 0.95236013 0.95375945 0.95509035 0.95634837 0.9575527
 0.95869752 0.95978613 0.96082785 0.96182127 0.96277573 0.96368659
 0.96455682 0.96539024 0.96619295 0.9669617  0.96770154 0.96841116
 0.96909127 0.96974576 0.97037507 0.97097799 0.97156183 0.97212111
 0.97265794 0.97317396 0.97367259 0.97415146 0.974613   0.97505725
 0.97548377 0.97589414 0.97628887 0.97666868 0.97703349 0.97738494
 0.97772177 0.97804559 0.97835549 0.97865392 0.97893959 0.97921224
 0.97947398 0.97972293 0.97996145 0.98018772 0.98040276 0.98060723
 0.98080005 0.98098217 0.98115269 0.98131315 0.98146197 0.9815993
 0.98172652 0.98184122 0.98194526 0.98203734 0.98211745 0.98218581
 0.98224084 0.98228383 0.98231302 0.98232917 0.98233008 0.98231656
 0.98228819 0.98224242 0.98218066 0.98209967 0.98199961 0.98188018
 0.98173791 0.98157214 0.98137982 0.98116057 0.98090945 0.980622